# Finetune PE dataset for LTC

## Libraries

In [1]:
# %cd ..
# %rm -rf LLaMA-Factory
# !git clone https://github.com/hiyouga/LLaMA-Factory.git
# %cd LLaMA-Factory
# %ls
# !pip install -e .[torch,bitsandbytes]

In [2]:
# !pip uninstall -y pydantic
# !pip install pydantic==1.10.9 # 

# !pip uninstall -y gradio
# !pip install gradio==3.48.0

# !pip uninstall -y bitsandbytes
# !pip install --upgrade bitsandbytes

# !pip install tqdm
# !pip install ipywidgets
# !pip install scikit-learn

# Restart kernel afterwards.

In [1]:
import os
import ast
import json
import torch
import pickle
import pandas as pd
from tqdm.notebook import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report

In [4]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

## Load Dataset

In [5]:
os.getcwd()

'/Utilisateurs/umushtaq'

In [ ]:
# *** MODEL NAME ***

# base_model = "unsloth/llama-3-8b-Instruct-bnb-4bit"

with open("tmp.pkl", "rb") as fh:
        
        l = pickle.load(fh)
        base_model = l[0]
        train_dataset_name = l[1]
        test_dataset_name = l[2]

In [6]:
dataset_dir = os.path.join(os.getcwd(), "datasets")

# *** TRAIN DATASET NAME *** #

# train_dataset_name = "PE_LTC_train.json"
train_dataset_file = os.path.join(dataset_dir, train_dataset_name)

# *** TEST DATASET NAME *** #

# test_dataset_name = "PE_LTC_test.json"
test_dataset_file = os.path.join(dataset_dir, test_dataset_name)

## Fine-tune Model

In [8]:
# *** MODEL DIR ***
model_name = f"""{train_dataset_name.split(".")[0].split("train")[0]}{base_model.split("/")[1]}"""

train_file = os.path.join(os.getcwd(), f"""cli_files/{train_dataset_name.split(".")[0].split("train")[0]}{base_model.split("/")[1]}.json""")
output_dir = os.path.join(os.getcwd(), "models", model_name)

nb_epochs = 10 # 3

In [9]:
train_file

'/Utilisateurs/umushtaq/cli_files/PE_LTC_llama-3-70b-Instruct-bnb-4bit.json'

In [10]:
os.path.isfile(train_file)

True

In [11]:
dataset_info_line =  {
  "file_name": f"{train_dataset_file}",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output"
  }
 }

In [12]:
dataset_info_line

{'file_name': '/Utilisateurs/umushtaq/datasets/PE_LTC_train.json',
 'columns': {'prompt': 'instruction', 'query': 'input', 'response': 'output'}}

In [13]:
os.getcwd()

'/Utilisateurs/umushtaq'

In [14]:
# with open("LLaMA-Factory/data/dataset_info.json", 'r+') as fh:
#     file_data = json.load(fh)
#     file_data["persuasive_essays"] = dataset_info_line
#     fh.seek(0)
#     json.dump(file_data, fh, indent = 4)

In [ ]:
with open("LLaMA-Factory/data/dataset_info.json", "r") as jsonFile:
    data = json.load(jsonFile)

data["persuasive_essays"] = dataset_info_line

with open("LLaMA-Factory/data/dataset_info.json", "w") as jsonFile:
    json.dump(data, jsonFile)

In [15]:
args = dict(
  stage="sft",                           # do supervised fine-tuning
  do_train=True,
  model_name_or_path=base_model,         # use bnb-4bit-quantized Llama-3-8B-Instruct model
  dataset="persuasive_essays",           # use alpaca and identity datasets
  template="llama3",                     # use llama3 prompt template
  finetuning_type="lora",                # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  output_dir=output_dir,                 # the path to save LoRA adapters
  overwrite_output_dir=True,             # overrides existing output contents
  per_device_train_batch_size=2,         # the batch size
  gradient_accumulation_steps=4,         # the gradient accumulation steps
  lr_scheduler_type="cosine",            # use cosine learning rate scheduler
  logging_steps=10,                      # log every 10 steps
  warmup_ratio=0.1,                      # use warmup scheduler
  save_steps=3000,                       # save checkpoint every 1000 steps
  learning_rate=5e-5,                    # the learning rate
  num_train_epochs=nb_epochs,            # the epochs of training
  max_samples=2000,                       # use 500 examples in each dataset
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  quantization_bit=4,                    # use 4-bit QLoRA
  loraplus_lr_ratio=16.0,                # use LoRA+ algorithm with lambda=16.0
  fp16=True,                             # use float16 mixed precision training
  report_to="none"                       # discards wandb
)

In [16]:
json.dump(args, open(train_file, "w", encoding="utf-8"), indent=2)

In [17]:
os.getcwd()

'/Utilisateurs/umushtaq'

In [18]:
%cd LLaMA-Factory/
!set train_file = train_file

/Utilisateurs/umushtaq/LLaMA-Factory


/Utilisateurs/umushtaq/.conda/envs/llama-factory-notebook/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [19]:
train_file

'/Utilisateurs/umushtaq/cli_files/PE_LTC_llama-3-70b-Instruct-bnb-4bit.json'

In [ ]:
!llamafactory-cli train $train_file

06/07/2024 18:43:21 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
06/07/2024 18:43:21 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2108] 2024-06-07 18:43:21,623 >> loading file tokenizer.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-70b-Instruct-bnb-4bit/snapshots/df8fab159e49c133bb4f074baeff4b303614b6d1/tokenizer.json
[INFO|tokenization_utils_base.py:2108] 2024-06-07 18:43:21,624 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2108] 2024-06-07 18:43:21,624 >> loading file special_tokens_map.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-70b-Instruct-bnb-4bit/snapshots/df8fab159e49c133bb4f074baeff4b303614b6d1/special_tokens_map.json
[INFO|tokenization_utils_base.py:2108] 2024-06-0

## Inference on the fine-tuned model

In [ ]:
output_dir

In [ ]:
os.listdir(output_dir)

In [ ]:
# %cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path=base_model, # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path=output_dir,            # load the saved LoRA adapters
  template="llama3",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)


In [ ]:
model = ChatModel(args)

In [ ]:
with open(test_dataset_file, "r+") as fh:
    test_dataset = json.load(fh)

In [ ]:
test_prompts = []
test_grounds = []

for sample in test_dataset:
    test_prompts.append("\nUser:" + sample["instruction"] + sample["input"])
    test_grounds.append(sample["output"])

In [ ]:
test_predictions = []

for prompt in tqdm(test_prompts):

    messages = []
    messages.append({"role": "user", "content": prompt})

    response = ""
    
    for new_text in model.stream_chat(messages):
        #print(new_text, end="", flush=True)
        response += new_text
        #print()
    test_predictions.append({"role": "assistant", "content": response})

    torch_gc()

In [ ]:
# next(model.engine.model.parameters()).is_cuda

In [ ]:
# len(test_predictions)

In [ ]:
# test_predictions

In [ ]:
with open(os.path.join(output_dir, f"""PE_LTC_results_{nb_epochs}.pickle"""), 'wb') as fh:
    results_d = {"ground_truths": test_grounds,
                 "predictions": test_predictions    
        
    }
    pickle.dump(results_d, fh)

## Post-processing

In [ ]:
#os.getcwd()

In [ ]:
# %cd ..

In [20]:
# output_dir = 'models/PE_LTC_essay_wo_tags_llama-3-8b-Instruct-bnb-4bit'
# nb_epochs = 10

In [21]:
with open(os.path.join(output_dir, f"""PE_LTC_results_{nb_epochs}.pickle"""), "rb") as fh:
        
        results = pickle.load(fh)

In [22]:
# results

{'ground_truths': ['{"relation_types": ["Support", "Support", "Support", "Support", "Support", "Support"]}',
  '{"relation_types": ["Support", "Support", "Support", "Support", "Support", "Support"]}',
  '{"relation_types": ["Support", "Support", "Support", "Support", "Support", "Support"]}',
  '{"relation_types": ["Support", "Support", "Support", "Support", "Support", "Support"]}',
  '{"relation_types": ["Support", "Support", "Support", "Support", "Support", "Support"]}',
  '{"relation_types": ["Support", "Support", "Support", "Support", "Support", "Support"]}',
  '{"relation_types": ["Support", "Support", "Support", "Support", "Support", "Support"]}',
  '{"relation_types": ["Support", "Support", "Support", "Support", "Support", "Support"]}',
  '{"relation_types": ["Support", "Support", "Support", "Support", "Support", "Support"]}',
  '{"relation_types": ["Support", "Support", "Support", "Support", "Support", "Support"]}',
  '{"relation_types": ["Support", "Support", "Support", "Suppor

In [4]:
grounds = results["ground_truths"]

In [5]:
preds = results["predictions"]

In [7]:
len(grounds), len(preds)

(80, 80)

In [8]:
grounds = [json.loads(x)["relation_types"] for x in grounds]

In [9]:
preds = [x["content"] for x in preds]

In [10]:
preds = [json.loads(x)["relation_types"] for x in preds]

In [18]:
grounds

[['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support',

In [19]:
preds

[['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'],
 ['Support', 'Support', 'Support',

In [11]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    if len(x) != len(y):
        print(i)

In [12]:
def opposite(relation_type):
    
    if relation_type == "Support":
        return "Attack"
    else:
        return "Support"

In [13]:
def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds    

In [14]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    
    if len(x) != len(y):
        
        preds[i] = harmonize_preds(x, y)

In [15]:
LTC_preds = [item for row in preds for item in row]
LTC_grounds = [item for row in grounds for item in row]

In [16]:
# sanity check: 
len(LTC_preds) == len(LTC_grounds)

True

## Results

In [17]:
print(classification_report(LTC_grounds, LTC_preds, digits=3))

              precision    recall  f1-score   support

     Support      1.000     1.000     1.000       480

    accuracy                          1.000       480
   macro avg      1.000     1.000     1.000       480
weighted avg      1.000     1.000     1.000       480



In [ ]:
with open(f"""{output_dir}/classification_report.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(LTC_grounds, LTC_preds, output_dict=True), fh)

In [ ]:
del model

In [ ]:
torch.cuda.empty_cache()